In [1]:
import requests
import pandas as pd
import time

def get_nearby_places(api_key, location, radius=500, place_types=None):
    """
    Function to get a list of nearby places within a specified radius.
    
    :param api_key: Your Google Places API key.
    :param location: A tuple of latitude and longitude (lat, lng).
    :param radius: Radius in meters within which to search for places.
    :param place_types: A list of place types to search for.
    :return: DataFrame containing details of nearby places.
    """
    if place_types is None:
        place_types = [
            'Supermarket', 'Store', 'Restaurant', 'Pharmacy', 'Laundry', 'doctor', 'Dentist', 'Cafe', 'Bakery', 'PT', 'CV', 'Clinic'
        ]
        
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    all_places = []
    
    for place_type in place_types:
        params = {
            'location': f"{location[0]},{location[1]}",
            'radius': radius,
            'keyword': place_type,
            'key': api_key
        }
        
        while True:
            response = requests.get(url, params=params)
            results = response.json().get('results', [])
            all_places.extend([{
                'name': result.get('name'),
                'address': result.get('vicinity'),
                'location': result.get('geometry', {}).get('location'),
                'type': place_type
            } for result in results])

            next_page_token = response.json().get('next_page_token')
            if not next_page_token:
                break
            
            # Wait a few seconds before requesting the next page
            time.sleep(2)
            params['pagetoken'] = next_page_token
    
    return pd.DataFrame(all_places)

# Google Places API key
API_KEY = ''

# Specific location (latitude, longitude)
location = (-6.1455356931012375, 106.83468846256453)  #

# Get nearby places within 10km radius
places_df = get_nearby_places(API_KEY, location)

# Display the DataFrame
print(places_df)

# Export the DataFrame to an Excel file
output_file = 'nearby_places.xlsx'
places_df.to_excel(output_file, index=False)
print(f"Data has been exported to {output_file}")


                                                  name  \
0                                             Ramayana   
1                      Agen Crital X Nassa Pisang batu   
2                                TTS Mangga Dua Square   
3                                           Toko Rizky   
4                                          TOKO BERKAH   
..                                                 ...   
283  Klinik Spesialis Bedah Umum dan Bedah Plastik ...   
284                                   Mizz beauty skin   
285                                Healthy Kids Clinic   
286                            Oriskin Kartini Jakarta   
287  Klinik Spesialis Andrologi - Klinik Utama Pandawa   

                                               address  \
0                VR4Q+75R, RT.16/RW.2, Pabuaran, Bogor   
1    Jl. Pisang Batu No.22, RT.4/RW.10, Mangga Dua ...   
2                     11, RT.11/RW.6, Ancol, Jkt Utara   
3    Jl. Hidup Baru 4 No.207 8, RT.6/RW.2, Pademang...   
4    Jl. Pade